In [68]:
import pandas as pd
import numpy as np

In [69]:
og_df = pd.read_csv('data/Structured/data.csv')
print(og_df.shape)

(44000, 376)


C:\Users\albin\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3442: DtypeWarning: Columns (296,297,298,299,300,301,302,303,304,305,306,307,321,322,323,324) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [70]:
df['Demand'] = pd.to_numeric(df['TOTALDEMAND'], errors ='coerce')
df['Price'] = pd.to_numeric(df['RRP'], errors ='coerce')

KeyError: 'TOTALDEMAND'

In [ ]:
df.drop(['TOTALDEMAND'], axis=1, inplace=True)
df.drop(['REGION'], axis=1, inplace=True)
df.drop(['SETTLEMENTDATE'], axis=1, inplace=True)
df.drop(['RRP'], axis=1, inplace=True)
df.drop(['PERIODTYPE'], axis=1, inplace=True)
df.drop(['Date Time'], axis=1, inplace=True)

df

KeyError: "['TOTALDEMAND'] not found in axis"

In [71]:

df = og_df.dropna(axis=1, how='all')

df = og_df.dropna(axis=1, how='all')
df = df.fillna(method='ffill')
df = df.fillna(method='bfill')
for e in df.columns:
    if df[e].nunique() == 1:
        df = df.drop(e, axis=1)
# df = df.drop('SE2', axis=1)
# df = df.drop('SE3', axis=1)
df = df.drop('SE4', axis=1)
date_time = pd.to_datetime(df.pop('Date'), format='%Y.%m.%d %H:%M:%S')
df = df.loc[:, ['SE1', 'SE2', 'SE3']]
# df = df.iloc[:,0:10]
print(df.shape)
print(labels.shape)
print(df.head())
# tf.convert_to_tensor(df)

(44000, 3)
(44000,)
      SE1     SE2     SE3
0  258.86  258.86  258.86
1  259.85  259.85  259.85
2  256.60  256.60  256.60
3  242.84  242.84  242.84
4  243.23  243.23  243.23


In [72]:
X0=[]
X1=[]
X2=[]
y=[]

In [73]:
for i in range (0, df.shape[0]-48):
    X0.append(df.iloc[i:i + 48, 0])
    X1.append(df.iloc[i:i + 48, 1])
    X2.append(df.iloc[i:i + 48, 2])
    y.append(df.iloc[i + 48, 0])

In [74]:
X0, X1, X2, y = np.array(X0), np.array(X1), np.array(X2), np.array(y)


In [75]:
y.shape


(43952,)

In [76]:
y=np.reshape(y, (len(y),1))
y.shape

(43952, 1)

In [77]:
X = np.stack([X0, X1, X2], axis=2)
X.shape


(43952, 48, 3)

In [78]:
X_train, X_test = X[:-480], X[-480:]
y_train, y_test = y[:-480], y[-480:]

In [79]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras import optimizers

In [80]:
model = Sequential()
# set return requesnce = true, if we are going to have another LSTM layer
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(50, activation='relu'))
model.add(Dense(1))

In [81]:
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 48, 50)            10800     
                                                                 
 lstm_7 (LSTM)               (None, 50)                20200     
                                                                 
 dense_6 (Dense)             (None, 50)                2550      
                                                                 
 dense_7 (Dense)             (None, 1)                 51        
                                                                 
Total params: 33,601
Trainable params: 33,601
Non-trainable params: 0
_________________________________________________________________


In [82]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

filepath = 'models/{epoch:02d}-{loss:.4f}-{val_loss:.4f}-{mae:.4f}-{val_mae:.4f}.hdf5'

callbacks = [EarlyStopping(monitor='val_loss', patience=20),
             ModelCheckpoint(filepath, monitor='loss', save_best_only=True, mode='min')]

In [83]:
optimizers.SGD(momentum=0.9)
model.compile(optimizer='SGD', loss='mse', metrics=['mae'])

In [84]:
model.fit(X_train, y_train, validation_split=0.2, epochs=1000, callbacks=callbacks, batch_size=16)


Epoch 1/1000
1564/2174 [====================>.........] - ETA: 19s - loss: 38019018752.0000 - mae: 10204.8486

KeyboardInterrupt: 

In [ ]:
MSE, MAE = model.evaluate(X_test, y_test)
print(MSE, MAE)

15/15 [==============================] - 1s 18ms/step - loss: 13933.3174 - mae: 55.3051
13933.3173828125 55.30514144897461


In [ ]:
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
predictions

15/15 [==============================] - 1s 16ms/step


NameError: name 'scaler' is not defined